In [0]:
# %run ../00_config

In [0]:
print("顧客レビューのCSVファイルを作成します...")

In [0]:
# 結果データセットを表示するか否か
# DISPLAY_ON = True     # ON
DISPLAY_ON = False    # OFF

##### 1. 共通関数

In [0]:
import pandas as pd
from typing import Dict, List, Tuple

def create_reviews(
    review_category: str,
    review_items: Dict[str, List[Tuple[str, int]]],  # (テキスト, スコア)
    rating_col: str,                                 # 追加: カラム名を指定
    start_id: int = 1
) -> pd.DataFrame:
    """
    review_items 例:
        {
          "negative": [("テキスト1", 4), ("テキスト2", 3), ...],
          "positive": [("テキストA", 9), ("テキストB", 10), ...]
        }
    rating_col 例:
        "rating_ife", "rating_seat_comfort", ...
    """

    # ネガティブ
    reviews = [
        {
            "sentiment": "ネガティブ",
            "review_category": review_category,
            "review_text": txt,
            rating_col: score
        }
        for txt, score in review_items["negative"]
    ]

    # ポジティブ
    reviews += [
        {
            "sentiment": "ポジティブ",
            "review_category": review_category,
            "review_text": txt,
            rating_col: score
        }
        for txt, score in review_items["positive"]
    ]

    # 連番 ID
    for idx, review in enumerate(reviews, start=start_id):
        review["id"] = idx

    # 列順: 動的に組み立てる
    cols = ["id", "sentiment", "review_category", "review_text", rating_col]
    return pd.DataFrame(reviews)[cols]

##### 2-1.機内エンターテイメント

In [0]:
# レビューテキスト
review_items = {
    "negative": [
        ("Bricks Airwaysの機内エンタメコンテンツは、選択肢が多すぎて逆に迷ってしまいます。シンプルに選びやすくしてほしいです。", 3),
        ("ニューヨークへの一人旅。深夜便で寝付けず映画に期待したのに、半分も進まないうちに再生が止まって最初から。結局6時間ひたすら時計を見て過ごした。", 1),
        ("南米から成田までの15時間、スクリーンが真っ暗のまま。相談しても “満席で席の移動は無理” の一点張り。支払った金額を返して欲しい。。。", 1),
        ("娘の大学留学先に向かうために利用しました。海外は初めてで、できれば効率的な移動方法や地域特化のTipsなど、もっと現地情報が充実していると嬉しいです。", 2),
        ("深夜発シンガポール便、フライト前に子ども用にダウンロードした映画が再生不可。キッズ向けチャンネルは英語吹替のみで、5歳児が飽きて椅子を蹴り始め地獄と化した。", 2),
        ("欧州出張の準備資料を見ながらBGM代わりにジャズを流したかったのに、プレイリストが『最新K-POP』か『90年代ロック』のみ。静かな曲を探して30分無駄にした。", 2),
        ("60代の父を伴う家族旅行。メニューもアイコンも小さくて読めず、父は操作を諦め窓側で腕組み。あと8時間もあると言ったらため息をつかれた。", 2),
        ("夜行便でしたが、ゲームコントローラが壊れて“スタート”が押せずタイトル画面ループ。CAさん『また戻って来ます』で放置。結局紙の安全カードを眺めていた。", 2),
        ("ベトナム出張帰り、機内のドキュメンタリーが全て2年前と同じラインナップ。仕事の疲れを癒やすどころか déjà-vu で余計にぐったりした。", 2),
        ("プレエコ席なのにスクリーン解像度が荒く、字幕がぼやけて読めず目が痛い。途中でギブアップしてスマホ読書に切り替えたが、照明が暗くて頭痛がした。", 2),
        ("ロンドン行き。ミュージックチャンネルで邦楽が1アルバムだけ、後は聞いたことのないインディーズ。結局ホワイトノイズを流して寝るしかなかった。", 2),
        ("飛行中の12時間、途中で映画の音声が左チャンネルだけになり臨場感ゼロ。ヘッドホン交換しても改善せず、映像は諦め寝返りを打ち続けて腰を痛めた。", 2),
        ("映像の画質が悪く、何度も再起動しないと視聴できなかった。飛行機内の設備管理が甘いです。", 2),
        ("コンテンツ更新が遅く、同じ映画ばかり見せられて飽きました。もっと新しいものを追加してほしいです。", 2),
        ("機内エンタメの音声が途切れがちで、映画に集中できませんでした。改善をお願いしたいです。", 2),
        ("現地情報のコーナーが、すでにネットで調べた内容とほとんど変わらず、あまり役立たなかったです。", 2),
        ("映像が常にカクカクしていて、快適に視聴できませんでした。改善の余地ありです。", 2),
        ("英語字幕が不完全で、映画を楽しめませんでした。字幕の品質をもっと向上させてください。", 2),
        ("ゲームのバリエーションが少なすぎて、飽きてしまいました。もっと多様なゲームを提供してほしいです。", 3),
        ("座席のタッチスクリーンが全く反応せず、何度も試す羽目に。これではエンタメどころではありません。", 2),
        ("映画を途中で止めても、再開する時に前の場所に戻らないため、非常に不便でした。", 2),
        ("知らないうちに機内のWi-Fiが切れて、動画のストリーミングが途切れてしまった。サービスの安定性が欠けています。", 2),
        ("機内の広告が多すぎて、映画を楽しんでいる最中でも何度も中断されてストレスでした。", 2),
        ("機内エンタメのコンテンツに日本語のものが少なく、英語が苦手な私には全く楽しめませんでした。", 2),
        ("スポーツイベントの中継があると思っていたのに、全くありませんでした。非常に残念です。", 2),
        ("キッズ向けコンテンツが少なく、大人向けのものばかりで子どもが退屈していました。もっと多様化してほしいです。", 2),
        ("映画を選ぶ際に、ジャンルごとに絞り込む機能がないので、見たい映画を探すのに時間がかかりすぎました。", 2),
        ("機内のゲームが全く面白くなく、退屈していました。もっとエンタメ性の高いゲームを導入してほしいです。", 2),
        ("コンテンツの読み込みが非常に遅く、乗り物酔いしている最中に待たされるのが辛かったです。", 2),
        ("映像が壊れていて、音声と合わず映画が楽しめませんでした。設備の整備が必要です。", 1),
        ("シートの音量がバラバラで、快適に映画を楽しめませんでした。", 2),
        ("字幕が小さくて、視覚的に非常に疲れました。もう少し大きくしてほしいです。", 2),
        ("初めての海外旅行で期待したが、ほぼ旧作映画やドラマばかりで見たことあるものばかり。探すのも億劫になり暇つぶし不足が辛かった。", 2),
        ("いろんな作品を見ようと思っていたのに、ディスプレイのタッチパネルが反応せず、貯金を貯めて初めの旅行だったのに残念です・・・。", 1),
        ("乗り継ぎで 4フライト連続Bricksを使ったら、エンタメが全便まったく同じ。月に5本も乗る常連には退屈すぎるラインナップ。", 2),
        ("海外出張で利用。できれば現地情報（特に移動手段やレストラン情報など）を知りたかったがあまり参考になるものがなかった。もっと現地の便利情報を増やしてほしい。", 2),
        ("左手が不自由なのでリモコン一体型のスクリーンは操作困難。バリアフリー設計がまるで考慮されていないと感じた。", 1),
        ("夜行便で画面を暗くしても周囲の光漏れが酷く、眠る乗客に気を使って消したら何もすることがない。読書灯も故障で完全に暇だった。", 2),
        ("写真整理をしようと USB 充電ポートに接続したが通電せず、バッテリー切れ。Wi-Fi も遅く作業できず11時間が無為に過ぎた。", 2),
        ("開発合宿帰りにコードレビューをしようとしたがIDEがクラッシュするほどの低帯域。『ビジネスクラス向け高速 Wi-Fi』の看板に偽りあり。", 1),
        ("最新アニメを期待していた高校生の息子が、シーズン1の第1話だけで構成されたリストを見て激萎え。以後ずっと『暇すぎる』と文句を言われ続けた。", 2),
        ("夜勤明けで羽田から搭乗。仮眠しようとしたらスクリーンの明るさ調整が利かず、顔をタオルで覆う羽目に。結局ほとんど眠れず疲れだけが残った。", 2),
        ("成田発の深夜便、学生最後の卒業旅行だったのに、ホラー以外の新作映画がゼロ。怖がりの友人は仕方なく目をつぶって音楽だけ聴いていた。", 2),
        ("羽田発プレミアムエコノミーでPC作業予定だったが、USBポートが通電せずバッテリー切れ。『到着後のプレゼン資料』を作れず冷や汗もの。", 1),
        ("親子3世代旅行で利用。キッズ向けゲームが3種類しかなく、小2の息子が30分で飽きてしまい以後ずっと『まだ着かないの？』攻撃。", 2),
        ("帰省ラッシュで満席の便、隣席の荷物がはみ出していてリモコンが取り出せず操作不能。上映中の映画も途中停止できずトイレを我慢することに。", 2),
        ("長距離フライトで字幕をオンにしたら日本語と英語が同時表示され画面が文字だらけ。設定で片方だけ選べず、目が疲れて頭痛になった。", 2),
        ("初海外の母は小柄でタッチパネルに手が届かず、映画を選ぶたびに私が身を乗り出して操作。席が狭いので腰痛が悪化した。", 2),
        ("離陸直後から終始ノイズが乗った音声でセリフが聞き取れず、洋画なのに字幕も消せない。逆に集中して眠れなくなった。", 1),
        ("2歳児用に用意されたアニメが1話5分×4本だけ。残り9時間は寝かしつけとぐずり対応で親がヘトヘト。", 2),
        ("夜行便でBGMチャンネルを探したら全曲ボーカル入り。環境音やクラシックがなく、耳栓で無音にした方がまだ寝られた。", 2),
        ("乗り継ぎで時間が読めないので途中停止できるドラマを選んだが、再開時に別エピソード冒頭へ飛ぶバグでストーリーが混乱。", 2),
        ("羽田着陸前に機内カメラ映像を流すはずが、真っ白な画面のまま終了。初めての夜景を楽しみにしていた弟がっかり。", 2),
        ("視覚障がいのある友人と利用したが、音声ガイド対応の映画が1本もなく、機内5時間ほとんど会話もできず申し訳なく感じた。", 1),

        # 1〜5：深夜便・家族旅行・操作系トラブル
        ("羽田発の夜行便でスクリーンの明るさ調整バーが壊れており眩しくて眠れず。CAに相談しても『仕様です』だけで耳栓とアイマスクを渡されて終わり。", 2),
        ("成田発の長距離便、映画のエンドロールが流れるたびに強制的に広告へジャンプ。作品の余韻が台無しで映画好きとしてはがっかり。", 2),
        ("仕事資料を開きながらBGMを流そうとしたら、エンタメアプリを最小化した瞬間に音が切れる仕様。仕事も娯楽も中断されストレス。", 2),
        ("羽田発の帰国便、Bluetooth対応と聞いていたのに自分のノイキャンイヤホンが接続不可。有線ヘッドホンは常にノイズ混じりで頭痛がした。", 1),
        ("成田発深夜便、座席ポケットのリモコンがベタついてボタンが埋没。触るのも嫌で結局自前のオーディオブックで凌いだ。", 2),

        # 6〜10：子連れ・高齢者・バリアフリー
        ("母子2人旅だがキッズ向けプレイリストに日本語の子守歌や絵本朗読がなく、到着まで『まだ？』攻撃を受け続け疲労困憊。", 2),
        ("初めての一人旅。映画検索のキーボードがABC固定で日本語タイトルを探すのに毎回スクロール地獄。昭和のビデオデッキみたい。", 2),
        ("羽田着の早朝便、環境音を流して寝ようとしたら突然機内ショッピング広告が大音量で割り込み、心臓が飛び出るほど驚いた。", 2),
        ("エコノミー最前列で脚は楽だったがスクリーンが壁収納式で離陸後20分出せず。暇すぎて機内誌を3周読んだ。", 3),
        ("成田発、障がい者向けの音声ガイドがあるとHPにあったが実際は英語のみ。視覚障害の兄はまったく使えず静かに落ち込んでいた。", 1),

        # 11〜15：情報の古さ・UI/UX・デバイス連携
        ("ワーケーション帰国便。ニュースチャンネルは1週間前の録画、株価情報も古く役立たず。Wi-Fiも遅くて調べ物ができずストレス。", 2),
        ("羽田行き夜便。電子書籍サービスで漫画を開いたらページ送りが0.5秒遅延。テンポ悪すぎて途中で断念。", 2),
        ("ハードロック好きだが音楽カテゴリーはポップとクラシック中心。検索バーもなく１曲探すのに15分。結局スマホに切り替えた。", 2),
        ("親子３世代旅行。祖母が期待していた昔の邦画ゼロ、吹替作品も少なく結局窓の外を眺めて終わり。", 2),
        ("成田発ビジネスクラスなのにUSB-Cポートが映像リンク専用で充電不可。タブレットの残量を気にしながらの映画鑑賞は落ち着かない。", 2),

        # 16〜20：機材差・シリーズ不足・システム不具合
        ("同僚と同じ列なのにスクリーンの世代が違いイヤホンジャック径もバラバラ。ケーブル貸し借りできず地味にストレス。", 2),
        ("羽田行き。連続ドラマがシーズン途中で４話で終わり。着陸後すぐ結末を検索する羽目に。モヤモヤが残った。", 2),
        ("新作邦画を見ていたら途中から字幕が別作品に入れ替わり、クライマックスで役者名ロールが続いて完全に興醒め。", 1)
    ],

    "positive": [
        ("Bricks Airwaysの機内エンタメは種類が豊富で、長時間のフライトも退屈せずに過ごせました。次回も利用したいです。", 5),
        ("映像の画質が非常に高く、映画を最高の状態で楽しめました。素晴らしい設備です。", 5),
        ("機内エンタメのゲームが面白く、暇な時間を楽しく過ごせました。もっとたくさんのゲームをプレイしたいです。", 5),
        ("音楽のプレイリストが豊富で、リラックスできる時間を提供してくれました。次回も楽しみにしています。", 3),
        ("現地情報が非常に役立ち、観光地やレストランの選択肢が広がり、旅行がより充実しました。", 5),
        ("映画を途中で止めても、再開するときにスムーズに前の位置に戻るので、使いやすかったです。", 5),
        ("子ども向けのアニメーション映画が充実しており、子どもが機内で退屈することなく過ごせました。", 5),
        ("映像が美しく、映画館で観るような臨場感を楽しめました。素晴らしい体験です。", 5),
        ("ブラウザ上でのストリーミングがスムーズで、何も気にせず映画を楽しむことができました。", 5),
        ("自分のNetflixアカウントと連携でき、家で観ているような感覚で映画を楽しめました。非常に便利でした。", 5),
        ("機内のWi-Fi接続が非常に安定していて、快適に映画や動画を楽しめました。次回も利用したいです。", 5),
        ("機内で提供されるエンタメコンテンツが多岐にわたり、毎回違った映画や番組を楽しめました。", 5),
        ("乗っている間、子どもが退屈しないように配慮されているので、家族旅行にも最適です。", 4),
        ("機内エンタメのインターフェースが直感的で使いやすく、ストレスなく映画や音楽を選べました。", 5),
        ("飛行機内のスクリーンが大きく、映画を観る際に視覚的な楽しさが倍増しました。", 5),
        ("機内で観た映画が、家で観るよりも圧倒的に高画質で、まるで映画館にいるような気分になりました。", 5),
        ("映画だけでなく、現地情報や天気予報も表示されていて、旅行先の準備がしやすかったです。", 5),
        ("乗っている間に観た映画が非常に面白く、退屈しらずでした。次回もBricks Airwaysを利用したいです。", 5),
        ("乗客が多くて少し混雑していましたが、映画や音楽を楽しむスペースは確保されていて快適でした。", 4),
        ("機内エンタメの選択肢が豊富で、長時間のフライトでも退屈せずに楽しむことができました。", 5),
        ("他の航空会社と比べてエンタメが充実しており、非常に便利でした。次回も利用します。", 5)
    ]
}


# create dataframe
df = create_reviews(
    review_category = "機内エンターテイメント",     # テーマ
    review_items    = review_items,             # レビューテキスト + スコア
    rating_col      = "rating_ife",             # カラム名
    start_id        = 1                         # idの開始番号
)

# DataFrame → Spark
s_ife_df = spark.createDataFrame(df)

# カウントアップ
current_cnt = 0     # 初期化
current_cnt += s_ife_df.count()

# print
print(f"単体カウント: {s_ife_df.count()}")
print(f"ここまでの総カウント: {current_cnt}")
print(s_ife_df.columns)
if DISPLAY_ON:
    display(s_ife_df)

##### 2-2.座席の快適さ

In [0]:
# レビューテキスト
review_items = {
    "negative": [
        ("座席が狭くて長時間のフライトがとても辛かった。隣の人とぶつかってしまい、快適ではありませんでした。", 1),
        ("座席のリクライニングがほとんど倒れず、リラックスできませんでした。改善を希望します。", 3),
        ("シートピッチが狭すぎて足が窮屈でした。もっと広い席にして欲しいです。", 1),
        ("席が硬くて、何時間も座っているのが苦痛でした。もっとクッション性のある座席がいいです。", 3),
        ("座席に座っていると、どうしても腰が痛くなります。もっと体に優しいデザインにしてほしい。", 3),
        ("隣の席との間隔が狭く、他の乗客との接触が気になりました。もう少しスペースが欲しいです。", 2),
        ("座席が古くて、あまり清潔感がなかった。座ったときにちょっとした不快感がありました。", 3),
        ("足元が狭すぎて、長時間のフライトではかなり窮屈に感じました。改善が必要です。", 1),
        ("背もたれの角度が調整できず、長時間座っていると背中が痛くなりました。", 3),
        ("座席に座っていると、長時間で肩が凝りやすかった。もっと快適なシートにしてほしい。", 2),
        ("座席のシートカバーが汚れていた。もっと清潔に保って欲しいと思います。", 3),
        ("隣の席が非常に近く、リラックスして座れませんでした。もっとスペースを広げてほしいです。", 3),
        ("シートの肘掛けが硬くて、長時間のフライトでは腕が痛くなりました。", 3),
        ("座席が前後に動かなくて、足元が窮屈で苦痛でした。改善が必要です。", 3),
        ("座席に座っていると、長時間で腰が痛くなります。もっと快適に座れるシートが必要です。", 3),
        ("シートのデザインが古臭く、もっとモダンで快適なものにしてほしいです。", 2),
        ("座席のリクライニングがスムーズでなく、身体に負担がかかりました。", 3),
        ("座席の間に物が多すぎて、足を伸ばすスペースがありませんでした。", 2),
        ("長時間座っていると座席が硬く感じ、腰が痛くなりました。", 1),
        ("シートベルトが短くて、身動きが取りづらかった。もっと余裕が欲しいです。", 2)
    ],
    "positive": [
        ("座席が広くて、長時間のフライトでも快適に過ごせました。十分に足元のスペースも確保されていて良かったです。", 4),
        ("リクライニングが心地よく、映画を見ながらリラックスできました。非常に快適でした。", 5),
        ("シートピッチが広く、足を伸ばして座ることができたので、快適に過ごせました。", 4),
        ("座席のクッションが柔らかく、長時間でも疲れにくかったです。次回もこの航空会社を選びたいです。", 4),
        ("背もたれが調整可能で、自分に合った角度にできるので非常に快適でした。", 4),
        ("さらに広い足元スペースがあり、足を完全に伸ばせてリラックスできました。次回も利用したいです。", 5),
        ("座席が非常に広くて、両隣の乗客とぶつかることなくゆったりと過ごせました。", 4),
        ("座席が柔らかく、リクライニングもスムーズで映画を観るのに最適な環境でした。", 4),
        ("肘掛けが調整可能で、自分の座り方に合わせて使えるのが良かったです。", 3),
        ("長時間座っても腰が痛くならず、非常に快適でした。もっとこういう座席を増やしてほしいです。", 5),
        ("足元が広く、移動も楽だったため、他の乗客のことを気にせずにリラックスできました。", 4),
        ("長時間のフライトでも座席がしっかりサポートしてくれて、快適に過ごせました。", 4),
        ("座席のヘッドレストが高くて、首の疲れを軽減できたのが良かったです。", 3),
        ("クッション性のあるシートで、座り心地が非常に良かった。リラックスして過ごせました。", 4),
        ("シートのリクライニング角度が広く、休憩中に快適に横になれました。", 5),
        ("座席のスペースが広く、隣の乗客との接触が少なくて非常に快適でした。", 4),
        ("座席が新しくて清潔感があり、長時間座っても不快感がありませんでした。", 3),
        ("足元に十分なスペースがあり、フライト中も足を自由に動かすことができて快適でした。", 3),
        ("座席のシートが温かく、寒さを感じることなく過ごせました。すごくありがたかったです。", 3),
        ("シートの前後に比較的余裕があって快適でした。航空機なので完璧ではないものの、他社より良いです。次回もぜひ利用したいです。", 3)
    ]
}

# create dataframe
df = create_reviews(
    review_category = "座席の快適さ",             # テーマ
    review_items    = review_items,             # レビューテキスト + スコア
    rating_col      = "rating_seat_comfort",    # カラム名
    start_id        = current_cnt + 1           # idの開始番号
)

# DataFrame → Spark
s_seat_df = spark.createDataFrame(df)

# カウントアップ
# current_cnt = 0     # 初期化
current_cnt += s_seat_df.count()

# print
print(f"単体カウント: {s_seat_df.count()}")
print(f"ここまでの総カウント: {current_cnt}")
print(s_seat_df.columns)
if DISPLAY_ON:
    display(s_seat_df)

##### 2-3.食事と飲み物

In [0]:
# レビューテキスト
review_items = {
    "negative": [
        ("ビジネス出張の途中で機内食が出されたが、冷めていて非常に不味かった。忙しい中での食事だったので、せめて温かいものが食べたかった。", 1),
        ("家族旅行で長時間のフライトだったが、食事が味気なく、子どもがほとんど食べなかった。もっと子ども向けのメニューが充実していると助かります。", 3),
        ("フライト中に何度も飲み物を頼んだが、なかなか届かず、乾いた喉を我慢する羽目になった。サービスが遅くて不快だった。", 2),
        ("私はダイエット中なので、低カロリーの食事を頼んだが、提供された食事が全く低カロリーでなく、選択肢の無さに失望した。", 2),
        ("長時間のフライトで、機内食を楽しみにしていたが、食事の量が少なく、物足りなかった。もう少しボリュームが欲しい。", 3),
        ("機内で提供された飲み物が温かった。暖かい飲み物をお願いしたが、冷めたコーヒーが出てきてがっかりした。", 2),
        ("出発からかなりの時間が経っても食事が提供されず、結局降りる寸前に出されて、急いで食べる羽目に。もっと計画的に出してほしい。", 3),
        ("フライトが非常に混雑していて、飲み物を頼んでもなかなか回ってこなかった。さらにお水がないと言われてかなり不安になった。", 1),
        ("出された機内食は、冷たくて硬く、まるで予算を削った感じがして非常に不愉快だった。", 3),
        ("出された料理が全体的に塩辛く、口に合わなかった。フライトが長いだけに、この食事で長時間過ごすのは辛かった。", 2),
        ("全然温かい飲み物が来なかった。せっかくリクエストしたのに冷めた飲み物を提供されたことにかなり驚いた。", 1),
        ("機内食が単調で、同じメニューが繰り返し出されて飽きてしまった。もっとバラエティに富んだメニューが欲しい。", 3),
        ("食事のアレルギー対応を頼んだのに、アレルギー物質が含まれているものが出されてしまった。安全性をもっと重視してほしい。", 1),
        ("食事の準備が遅く、ずっと待たされた。長時間フライトの中で、この待機時間がストレスになった。", 3),
        ("機内食が思った以上に脂っこく、消化に時間がかかってフライト後も気分が優れなかった。もっと軽めのメニューが欲しい。", 3),
        ("特に赤ちゃん用の食事が無くて、子どもが空腹で泣き続ける羽目に。子ども向けのメニューがあるともっと楽になる。", 2),
        ("フライト中に飲み物を注文したが、すぐに飲み物を持ってきてもらえなかった。そのため、かなりの時間乾ききっていた。", 3),
        ("座席で提供された食事は、まるでホテルのバンケット料理のようなものだった。フレッシュさが感じられなかった。", 2),
        ("ビジネスクラスなのに、機内食が非常にシンプルで、期待外れだった。もっと豪華なメニューを提供してほしい。", 1),
        ("機内食が冷めていて食べる気が失せた。飛行機でこんなに不味い食事を食べるのは初めてだった。", 3),
        ("長時間のフライト中に、機内食を楽しみにしていたが、量が少なく、すぐにお腹が空いてしまった。", 3)
    ],
    "positive": [
        ("機内食がとても美味しく、特にデザートが絶品だった。飛行機の中でこんなに満足した食事を食べたのは初めて。", 5),
        ("長時間のフライトだったが、食事が思ってたより美味しかったです。量もちょうど良く満たされました。", 4),
        ("飲み物のサービスがとても迅速で、乾ききった喉にちょうど良いタイミングで冷たい水が届き良かったです。", 4),
        ("フライト中に提供されたサラダが新鮮で美味しかった。食事が軽くて健康的で、長時間のフライトでも体が楽だった。", 4),
        ("機内の飲み物サービスが非常に親切で、コーヒーやジュースのリフィルもこまめにしてくれ、非常に助かりました。", 4),
        ("食事の量がちょうど良く、何も足りないと感じることなく、フライトを楽しむことができました。", 3),
        ("ビジネスクラスで提供された食事は、まるでレストランで食べるようなクオリティだった。見た目も美しく、食べるのが楽しみだった。", 5),
        ("機内食の温かさとフレッシュさが完璧で、こんなに美味しい料理をフライト中に食べられることに驚いた。", 5),
        ("飛行機で提供される食事がこんなにおいしいとは思わなかった。特にスープは温かくてホッとする味で、心まで温まった。", 4),
        ("フライトが長かったが、食事が3回しっかり提供されて、お腹いっぱいになり、快適に過ごせた。", 4),
        ("飲み物の種類が多く、どれも美味しくてリフレッシュできた。特に機内でのフルーツジュースはとても美味しかった。", 3),
        ("提供された食事がとてもヘルシーで、フライト中に体調が良く、ずっと元気に過ごせた。こんな食事が提供されるなんて素晴らしい。", 4),
        ("機内食の選択肢が豊富で、いろんな国の料理を楽しめた。フライト中に異文化を味わうことができ、嬉しかった。", 4),
        ("家族旅行で機内食を楽しみにしていたが、子ども向けの料理が特に美味しく、子どもたちが満足してくれた。", 4),
        ("機内の飲み物サービスがこまめで、長時間のフライトでも喉が乾くことがなかった。満足です。", 3),
        ("食事が美味しく、量も十分だったので、長時間フライトの疲れも和らいだ。", 4),
        ("機内で出されたワインが非常に美味しく、フライト中にリラックスした時間を過ごせた。", 4),
        ("提供された食事が非常に軽く、消化に負担をかけず、フライト後も快適に過ごせた。", 4),
        ("機内食が本当に美味しくて、フライト中にこんなにリラックスできるとは思わなかった。", 5),
        ("機内で観た映画が非常に面白く、退屈しらずでした。次回もBricks Airwaysを利用したいです。", 3),
        ("提供された食事がとてもヘルシーで、消化に優しく、フライト中も快適に過ごせた。", 4)
    ]
}


# create dataframe
df = create_reviews(
    review_category = "食事と飲み物",             # テーマ
    review_items    = review_items,             # レビューテキスト + スコア
    rating_col      = "rating_food_beverages",  # カラム名
    start_id        = current_cnt + 1           # idの開始番号
)

# DataFrame → Spark
s_foods_df = spark.createDataFrame(df)

# カウントアップ
# current_cnt = 0     # 初期化
current_cnt += s_foods_df.count()

# print
print(f"単体カウント: {s_foods_df.count()}")
print(f"ここまでの総カウント: {current_cnt}")
print(s_foods_df.columns)
if DISPLAY_ON:
    display(s_foods_df)

##### 2-4.価格に対する価値

In [0]:
# レビューテキスト
review_items = {
    "negative": [
        ("料金が高く感じたが、実際のサービスがそれに見合っていなかった。コストパフォーマンスが悪いと感じた。", 3),
        ("高額なチケット代を払っても、提供されるサービスが基本的なものであり、料金に見合った価値が感じられなかった。", 3),
        ("他の航空会社に比べて料金が高いが、サービスのクオリティが低いため、次回はもっとコスパの良い航空会社を選ぶかもしれない。", 3),
        ("価格に見合ったサービスを提供されると思っていたが、実際には不便で、サービス内容に満足できなかった。", 2),
        ("値段が高い割に、食事のクオリティが低く、全体的なコストパフォーマンスに疑問を感じた。", 2),
        ("チケット代が高いが、その分の価値が感じられない。次回は価格に見合うだけのサービスを求めたい。", 3),
        ("高額な料金を支払ったが、座席の快適さも中途半端で、他の航空会社と大差なく、コスパが悪かった。", 3),
        ("高い料金に見合ったサービスが期待できなかった。特にフライト中の快適さが不足していて、価格に見合わないと感じた。", 3),
        ("価格が高い割に、サービスの質が低く、最終的には高い料金を支払うことが無駄に思えた。", 1),
        ("料金が高い割に、提供されるアメニティが非常に基本的で、もっと高品質なサービスを提供すべきだと感じた。", 3),
        ("他の航空会社より高い料金を払ったが、フライト中の体験が期待外れだった。次回はよりリーズナブルな選択肢を選ぶかもしれない。", 2),
        ("価格に対して期待したよりもサービスがシンプルで、支払った金額に見合った価値が提供されなかった。", 3),
        ("高額な料金を支払ったにもかかわらず、機内食やサービスに不満があり、全体的にコスパが悪いと感じた。", 3),
        ("料金が高い割に、座席の広さやリクライニングの機能に不満があり、長時間のフライトには不向きだと思った。", 3),
        ("価格が高くて、サービスや座席の快適さがそれに見合わなかった。支払った金額が無駄に思えた。", 4),
        ("高額な航空券を買ったが、飛行機内でのエンタメが古く、全体的に料金に見合う価値がないと感じた。", 2),
        ("料金に見合ったサービスではなく、特にフライト中のサービスが粗末で、期待していた価値が得られなかった。", 1),
        ("高い料金に見合うような特別なサービスが提供されず、非常にがっかりした。", 2),
        ("価格に対してサービスが過剰にシンプルで、他の低価格の航空会社と比較しても大差がないと感じた。", 3),
        ("料金が高いわりに、サービスやエンタメの質が低く、次回は価格重視で別の航空会社を選ぶかもしれない。", 4)
    ],
    "positive": [
        ("料金は高かったが、その分、サービスが素晴らしく、非常に満足できた。価格に見合った価値が提供されていた。", 5),
        ("高い料金を払ったが、その分の価値が十分に感じられた。特に快適な座席と質の高いサービスに満足した。", 5),
        ("料金が高いが、それに見合った素晴らしいサービスと快適さが提供された。次回もぜひ利用したい。", 5),
        ("高額なチケット代でも納得できるサービスだった。特にフライト中の食事やエンタメが非常に充実していた。", 4),
        ("価格に見合った素晴らしい体験を提供してくれた。座席の広さやサービスが全て素晴らしく、快適なフライトだった。", 4),
        ("高い料金でも、それだけの価値があるサービスが提供された。長時間のフライトでも快適に過ごせた。", 4),
        ("料金は少し高いが、機内サービスや座席の快適さ、エンタメが素晴らしく、全体的に非常に満足した。", 4),
        ("高額なチケット代でも十分に納得できるサービス。機内の快適さと食事、飲み物の質が良かった。", 4),
        ("高い料金を払った価値があり、快適な座席、優れたサービス、そして美味しい食事に満足した。", 5),
        ("高いチケット代でも快適に過ごせる、リラックスできる環境を提供してくれる航空会社で、料金に見合うサービスだと感じた。", 3),
        ("値段が高かったが、サービスの質が高く、映画やゲームなどのエンタメが非常に充実していたので満足した。", 4),
        ("料金が高いが、それに見合った優れたサービスが提供され、フライト全体が快適だった。", 3),
        ("価格は高いが、それに見合った素晴らしい快適さと、航空会社の対応が良かった。", 3),
        ("高い料金を支払ったが、満足度が高かった。特に機内食やドリンクのクオリティが非常に良かった。", 4),
        ("料金に見合った素晴らしいサービスがあり、非常に快適なフライトとなった。コスパが良いと感じた。", 4),
        ("高額なチケット代でも、その分だけ価値があった。サービス、座席、食事、全てが素晴らしく、次回も利用したい。", 5),
        ("料金が高かったが、その分だけ快適さが充実しており、全体的に大満足のフライトだった。", 3),
        ("高いチケット代に見合うだけの価値があり、特に座席の広さとエンタメの質が素晴らしかった。", 3),
        ("料金が高いが、それに見合った素晴らしいサービスと快適なフライト体験ができ、納得できた。", 4)
    ]
}

# create dataframe
df = create_reviews(
    review_category = "価格に対する価値",          # テーマ
    review_items    = review_items,             # レビューテキスト + スコア
    rating_col      = "rating_value_for_money", # カラム名
    start_id        = current_cnt + 1           # idの開始番号
)

# DataFrame → Spark
s_value_df = spark.createDataFrame(df)


# カウントアップ
# current_cnt = 0     # 初期化
current_cnt += s_value_df.count()

# print
print(f"単体カウント: {s_value_df.count()}")
print(f"ここまでの総カウント: {current_cnt}")
print(s_value_df.columns)
if DISPLAY_ON:
    display(s_value_df)

##### 2-5.機内Wi-Fiと接続性

In [0]:
# レビューテキスト
review_items = {
    "negative": [
        ("出張中にフライト時間を有効活用しようと思っていたが、Wi-Fi接続が不安定で、仕事が全く進まなかった。コストを払ったのに、あんなに使えないネット接続はあり得ない。", 1),
        ("飛行機内でネットが使えないと、孫と連絡が取れないので困った。旅行中にWi-Fiを利用したいと思っていたのに、接続が全然ダメだった。", 2),
        ("フライト中に映画を見ようとしたけれど、Wi-Fiが途切れがちで何度も再接続を試みた。でも全然うまくいかない。Wi-Fiの質が低すぎる。快適に過ごしたかっただけなのに、ストレスが溜まった。", 2),
        ("価格が高いにも関わらず、Wi-Fiが遅くて使い物にならなかった。商談の準備ができると思っていたのに、結局まともにネットを使えず、非常に不便だった。", 2),
        ("機内Wi-Fiが全く使えなかったので、子どもと連絡が取れず非常に困った。旅行中にWi-Fiを利用したいと思っていたのに、まったく接続できなかった。", 2),
        ("ネットを使う仕事をしているのに、Wi-Fi接続が途中で切れて作業が進まなかった。お金を払って利用する価値が全く感じられなかった。", 2),
        ("飛行機内でWi-Fiを使って重要なメールを送るつもりだったが、接続が悪すぎて何も送れなかった。次回のフライトはWi-Fiの質を重視したい。", 2),
        ("フライト中に子どもたちが退屈しないようにWi-Fiを使いたかったが、接続が遅くて全く役に立たなかった。旅行の重要な部分が台無しになった。", 3),
        ("Wi-Fiがすぐに切れてしまって、せっかくの長時間フライトで映画を楽しむことができなかった。値段に見合ったサービスではなかった。", 3),
        ("Wi-Fi接続が遅すぎて、仕事をするどころか、ネットサーフィンさえできなかった。フライト時間を無駄に過ごした気分。", 3),
        ("接続したはいいが、映画を観るどころか、ウェブページを開くことすらできなかった。何のためにお金を払ったのか疑問。", 1),
        ("飛行機のWi-Fiが使えると思ったら全く使えず。時間を無駄にしただけでなく、疲れも倍増した。", 1),
        ("映画や音楽を楽しむ予定だったが、Wi-Fiが使えず全く何もできなかった。長時間のフライトでこれは最悪。", 2),
        ("Wi-Fiのサービスに期待していたが、つながらず、ビジネスメールを送信するのに大きな問題が生じた。", 2),
        ("機内Wi-Fiがあまりにも遅く、せっかくの旅行中に重要な情報を探すことができなかった。お金を払う意味がなかった。", 3),
        ("接続も遅く、途中で切れることもあり、非常に使いづらかった。フライト中の仕事に支障をきたした。", 3),
        ("他の航空会社と比較してもWi-Fiが最悪だった。接続中に切れてしまい、仕事が全然進まなかった。", 1),
        ("短時間でも接続が切れるので、オンラインで映画を観ることができず、非常に不満だった。", 3),
        ("予約したWi-Fiサービスが機能せず、代わりにオフラインの時間を過ごすことを余儀なくされた。", 1),
        ("飛行機の中でネットを使う予定だったが、接続が遅く、ほとんど何もできなかった。サービスの向上を期待している。", 3),
        ("Wi-Fiの質が悪すぎて、オンラインでの仕事が全くできなかった。あの値段を払った意味がなかった。", 2)
    ],
    "positive": [
        ("Wi-Fi接続が非常にスムーズで、フライト中も仕事が進められて助かった。特にプログラミングをしていたので、スピーディな接続は重要だった。", 5),
        ("旅行中に家族と動画通話をしたかったが、Wi-Fi接続がとても快適で、ずっと繋がっていられた。安心して楽しめた。", 4),
        ("Wi-Fi接続が速く、仕事の合間にリモート会議も問題なく参加できた。フライト中にこんなに効率よく過ごせるとは思わなかった。", 4),
        ("飛行機内でソーシャルメディアをチェックしながら、旅行先の情報を調べるのにWi-Fiがすぐに繋がり、非常に便利だった。", 4),
        ("インターネットがすぐに使え、旅行中に調べ物や仕事ができた。ストレスなくフライト時間を有効に活用できて満足。", 4),
        ("フライト中に重要な資料をオンラインで確認したかったが、Wi-Fiがスムーズに繋がり、何の問題もなく作業を進められた。すごく便利だった。", 4),
        ("Wi-Fiが安定していて、飛行機内でも友達とゲームをしたり、映画を観たりと、退屈せずに過ごせた。次回も利用したい。", 4),
        ("飛行機内でWi-Fiを使って、旅行のプランを立てたり、現地の観光情報を調べたりできて非常に便利だった。次回も利用したい。", 4),
        ("リモートワークをしているので、フライト中でも仕事ができるのは非常に助かる。Wi-Fi接続がスムーズで、ストレスなく作業を進められた。", 4),
        ("フライト中にオンラインで映画を観たかったが、Wi-Fiの接続がとても速く、快適に視聴できた。長時間のフライトでも全く問題なかった。", 5),
        ("インターネット接続が速く、ビジネスでの打ち合わせをスムーズにこなすことができた。非常に便利だった。", 4),
        ("機内Wi-Fiが快適で、フライト中の暇つぶしにピッタリだった。次回のフライトもぜひ使いたい。", 3),
        ("飛行機内で快適にWi-Fiを利用できたおかげで、映画鑑賞やネットサーフィンがとても楽しめた。", 4),
        ("フライト中に簡単にネットにアクセスでき、事前に確認したいことも問題なく調べられて非常に便利だった。", 3),
        ("機内Wi-Fiの接続速度が早く、動画や音楽のストリーミングが途切れず楽しめた。", 4),
        ("リモートワークで使えるWi-Fiが飛行機内にあったおかげで、仕事の進捗が非常に良かった。", 4),
        ("長時間のフライト中、Wi-Fiが安定していて快適に過ごせた。次回もWi-Fiを使用して仕事をしたい。", 4),
        ("機内Wi-Fiが非常にスムーズで、途中で切れることなく、フライトの間中ずっと快適にインターネットを使えた。", 4),
        ("フライト中にWi-Fiが安定していて、SNSや映画を観るのに困ることなく過ごせた。非常に便利。", 4),
        ("機内Wi-Fiの速度が速く、フライト中でも家族と連絡を取り合えたので非常に助かった。", 4)
    ]
}

# create dataframe
df = create_reviews(
    review_category = "機内Wi-Fiと接続性",            # テーマ
    review_items    = review_items,                 # レビューテキスト + スコア
    rating_col      = "rating_wifi_connectivity",   # カラム名
    start_id        = current_cnt + 1               # idの開始番号
)

# DataFrame → Spark
s_wifi_df = spark.createDataFrame(df)

# カウントアップ
# current_cnt = 0     # 初期化
current_cnt += s_wifi_df.count()

# print
print(f"単体カウント: {s_wifi_df.count()}")
print(f"ここまでの総カウント: {current_cnt}")
print(s_wifi_df.columns)
if DISPLAY_ON:
    display(s_wifi_df)

##### 2-6.機内スタッフのサービス品質

In [0]:
# レビューテキスト
review_items = {
    "negative": [
        ("フライト中、飲み物のサービスをお願いしたが、スタッフが何度も通り過ぎてしまい、最終的に自分でカートまで取りに行く羽目になった。十分な人数がいたはずなのに、どうしてこんなに対応が遅いのか理解できなかった。", 3),
        ("機内で突如として隣の席の人が体調不良を訴えたが、スタッフの対応が遅すぎた。迅速に対応してほしいのに、声をかけても全く焦らず、何度も確認してからしか動いてくれなかった。非常に不安な時間だった。", 1),
        ("機内食を頼んだが、指定した食事が間違って提供され、さらにスタッフが謝罪するどころか、面倒臭そうに対応された。そんなミスがあっても、もう少し親切に対応してほしかった。", 2),
        ("子供がぐずり始めたときに、スタッフに助けを求めたが、無視された。もう少し子供連れに配慮して、優しく声をかけてほしいと思った。", 2),
        ("緊急時に、機内スタッフが何も指示を出さず、乗客は不安そうに過ごしていた。パニックに陥りそうな状況で、もっと冷静で確実な対応を期待していた。", 1),
        ("機内のスタッフがやたらと無愛想で、何かお願いするたびに不機嫌そうな顔をされて嫌な思いをした。航空会社として、もう少し笑顔と心遣いを大事にしてほしい。", 3),
        ("機内食を注文したが、メニューにない食事が届き、スタッフは特にお詫びもなく無言で去って行った。せっかく楽しみにしていた食事が台無しになり、とても残念だった。", 2),
        ("隣の席の乗客がトラブルを起こしていた際、スタッフの対応が非常に遅かった。もう少し迅速で効果的な対応をしてほしかった。フライト中の安全は最優先であるべきだと思う。", 1),
        ("フライト中に何度も飲み物を頼んだが、毎回遅れてやってきた。おかげで、せっかくリラックスしていたのに、ストレスが増してしまった。", 3),
        ("乗客に何か問題があった時に、スタッフが無関心な態度を取っていた。問題が解決するまで何も手を貸してくれず、無言で座っているだけだった。機内スタッフにはもう少し責任感を持ってほしい。", 2),
        ("私が座席で寝ていたとき、スタッフが私の足元を無理やり通ろうとして、非常に不快に感じた。少しの気配りや声掛けがあれば、もっと快適に過ごせたはず。", 3),
        ("機内で食事をした際、隣の席の乗客に食事をこぼしてしまい、スタッフはすぐに助けてくれるかと思ったが、対応が遅く、その後も謝罪やフォローは一切なし。問題が解決した後のスタッフの態度にも不満が残った。", 3),
        ("機内の温度が寒すぎたのでブランケットをお願いしたが、スタッフはすぐに持ってきてくれなかった。寒い中で何度も頼んだが、やっと届いたのはフライトの終盤だった。", 3),
        ("スタッフが忙しすぎるのか、笑顔や配慮を全く感じられなかった。忙しい中でも最低限の礼儀や気配りをしてほしいと思った。", 3),
        ("緊急の対応が必要だった際に、スタッフが指示を出してくれず、周囲の乗客が混乱していた。スタッフはもっと冷静に指導してくれるべきだった。", 1),
        ("私は手荷物を預けた後に、シートベルトの緩みを指摘したが、スタッフはとても無関心で、また座席のクッションが壊れていた際も無視され、対処してもらえなかった。", 2),
        ("長時間フライトで何度も飲み物をお願いしたが、やっと持ってきた飲み物が冷たすぎて、結局飲めなかった。サービスのクオリティに一貫性がないと感じた。", 3),
        ("乗客に対してフレンドリーな対応をしてほしかったが、何度も頼みごとをすると、明らかに不快そうな顔をされて返答が遅かった。", 3),
        ("食事を頼んだ時、私のアレルギーについて尋ねられることなく、アレルゲンが含まれているものが出されてしまった。もっと配慮が必要だと感じた。", 2),
        ("フライト中にスタッフが何度も間違えて注文を取ったり、提供したりした。少しの手間でサービスの品質が向上するのに、もったいなかった。", 3),
        ("客室乗務員が機内で話しかけてきたが、質問に対してあまり親切に答えてくれなかった。自分の要望をしっかり聞いてくれると、もっと安心できた。", 3),
        ("機内で席の変更を頼んだが、スタッフは全く協力的でなく、むしろ面倒くさそうに応対していた。航空会社としてのプロフェッショナリズムを感じなかった。", 3)
    ],
    "positive": [
        ("フライト中、乗務員が私のリクエストをすぐに対応してくれ、食事の選択肢を変えてくれたおかげで快適に過ごせた。常に笑顔で対応してくれて、温かい気持ちになった。", 4),
        ("座席の後ろに座っていた乗客が体調不良になった際、スタッフがすぐに動き出し、冷静に適切な対応をした。私も安心して過ごせた。", 5),
        ("子どもがぐずり始めたときに、スタッフがすぐにおもちゃや絵本を持ってきてくれた。予想以上に気配りが行き届いていて、感謝している。", 4),
        ("長時間のフライトで、飲み物や軽食をこまめに提供してくれて、スタッフの配慮に感動した。次回もこの航空会社を利用したい。", 4),
        ("緊急時、乗務員がすぐに冷静に対処してくれ、乗客全員に安心感を与えてくれた。非常に頼もしいと感じた。", 5),
        ("フライトの途中、スタッフが自分の誕生日を祝ってくれた。ケーキとシャンパンを持ってきてくれたことにとても感激した。", 5),
        ("飲み物のサービスを頼んだ際、飲み物が目の前に現れたとき、スタッフは非常にフレンドリーで、私の名前まで呼んでくれた。気配りが素晴らしかった。", 4),
        ("機内での食事を選ぶ際、スタッフが親切にアレルギーについて尋ねてくれ、食事の選択肢を変えてくれた。非常に安心感があり、心地よかった。", 4),
        ("機内のトイレで急に子どもが不安そうになったとき、スタッフがすぐに駆け寄り、優しく声をかけてくれた。素晴らしい対応に感謝している。", 4),
        ("座席のリクエストをした際、スタッフが快く対応してくれたおかげで、飛行機内での時間が非常に快適だった。プロフェッショナルな接客に満足。", 4),
        ("フライト中、何度も飲み物や食事をお願いしたが、スタッフはどんなリクエストにも笑顔で応じてくれ、非常にありがたかった。", 4),
        ("飛行機の揺れがひどくて不安だったが、スタッフがしっかりと私をサポートしてくれて、安心して過ごせた。素晴らしい対応。", 4),
        ("私の足元のスペースが狭かったが、スタッフがすぐに配慮してくれ、リクライニングを調整してくれた。", 3),
        ("乗務員が細かいところまで気を配ってくれ、映画を観るために快適な場所に座れるようにサポートしてくれた。", 3),
        ("フライト中、スタッフがいつも快適に過ごせるように気配りをしてくれていた。長時間でも全く不快に感じなかった。", 4),
        ("フライト中に数回、飲み物や食事を注文したが、どれもタイミング良く提供され、サービスが素晴らしいと感じた。", 4),
        ("機内でのサービスが迅速で、何度も頼んだことを丁寧に対応してくれた。安心感があり、全体的に非常に良かった。", 4),
        ("乗務員が出発前に安全確認をしっかりしてくれ、飛行中も状況を適切にアナウンスしてくれたので、非常に安心して過ごせた。", 4),
        ("機内食を注文した際、スタッフが食事の温度や配膳に気を使ってくれて、どんな細かな要望にも対応してくれた。", 4),
        ("フライト中、急な変更に対応してくれたスタッフの迅速な対応に感謝している。大きな不安を感じることなくフライトを終えることができた。", 4),
        ("機内での体調不良をスタッフに伝えたが、すぐに温かいお茶を持ってきてくれ、回復するまでずっと見守ってくれた。", 4),
        ("スタッフが飛行機の機内での役割をよく理解していて、いつでも頼んだことをきちんと処理してくれた。", 4)
    ]
}

# create dataframe
df = create_reviews(
    review_category = "機内スタッフのサービス品質",       # テーマ
    review_items    = review_items,                   # レビューテキスト + スコア
    rating_col      = "rating_cabin_staff_service",   # カラム名
    start_id        = current_cnt + 1                 # idの開始番号
)

# DataFrame → Spark
s_staff_df = spark.createDataFrame(df)

# カウントアップ
# current_cnt = 0     # 初期化
current_cnt += s_staff_df.count()

# print
print(f"単体カウント: {s_staff_df.count()}")
print(f"ここまでの総カウント: {current_cnt}")
print(s_staff_df.columns)
if DISPLAY_ON:
    display(s_staff_df)

##### 2-7.地上サービス

In [0]:
# レビューテキスト
review_items = {
    "negative": [
        ("旅行中、初めての場所に行くため不安だったが、スタッフの冷たい対応でさらに不安が募った。笑顔一つ見せず、案内も不十分だったため、空港での待ち時間が不愉快だった。", 3),
        ("旅行先で荷物が届かず、空港スタッフに問い合わせたが、返答が遅く、手続きも煩雑だった。結局、荷物が届くまでの数日間、非常に不便な思いをした。", 2),
        ("予想以上に混雑しており、VIPラウンジの席はいっぱいで、スタッフが座席の案内をしてくれなかった。長時間立っているしかなく、疲れた。", 3),
        ("セキュリティチェックで何度も並ばされ、待機時間が長く、乗り継ぎ時間がギリギリになってしまった。スタッフはあまり手際よく動かず、もっとスムーズな運営を望んだ。", 3),
        ("事前にオンラインチェックインをしていたにも関わらず、カウンターでの手続きに時間がかかり、結局フライトのギリギリになってしまった。待機している間、スタッフは誰も状況を説明してくれなかった。", 3),
        ("空港のタクシーサービスを利用するつもりだったが、手配ミスで長時間待つ羽目になった。スタッフの対応は不十分で、結局別の方法で移動せざるを得なかった。", 3),
        ("予定していたフライトが遅れ、空港で長時間待機することになったが、スタッフはなかなか案内を出さず、他の航空会社の方がはるかに適切に対応していた。", 3),
        ("ラウンジ内のテーブルが汚れており、飲み物のカップもそのままで放置されていた。スタッフに言っても対応が遅く、不快な思いをした。", 3),
        ("大きな空港で案内板が分かりにくく、スタッフに尋ねても曖昧な返事しかもらえなかった。結果的に、出発ゲートに到着するのがギリギリになった。", 2),
        ("荷物を受け取る際、他の乗客との混雑でぶつかり合いがありましたが、スタッフは対応せず、誰も問題を解決しなかった。もっと注意深く運営してほしい。", 3),
        ("荷物を受け取る際、破損しているのを発見。スタッフに伝えたが、謝罪すらなく、何の補償もなかった。非常に失望した。", 1),
        ("トイレの清掃が不十分で、使用後に不快な思いをした。空港のスタッフが清掃の頻度をもっと増やすべきだと感じた。", 3),
        ("最前列を指定したが、空港スタッフの案内が遅く、結局別の席を案内された。何度も確認したのに、結果的に不便な席に座らされる羽目になった。", 3),
        ("事前に特別食をリクエストしていたにもかかわらず、空港スタッフが注文を漏らしていた。到着時にクレームを入れたが、対応が遅く不満が残った。", 3),
        ("空港内の案内所が夜間に閉まっていたため、問題が発生しても対応してもらえなかった。乗り継ぎで困ったが、スタッフは何もしてくれなかった。", 3),
        ("チェックイン時に、誤って航空券が無効にされてしまい、スタッフのミスで余計な手間がかかった。航空券確認をもっと慎重に行ってほしい。", 2),
        ("空港での仕事に欠かせないWi-Fiが非常に遅く、スムーズに仕事ができなかった。空港スタッフに話したが、改善の意欲が見られなかった。", 3),
        ("チェックイン時、スタッフが私のフライト時間を誤って伝え、その結果、搭乗ゲートに到着した際に慌ててしまった。もっと正確な情報を提供してほしい。", 3),
        ("空港内のトイレが使用不可になっていたが、スタッフからの事前の知らせがなく、利用する際に不便だった。案内が不足していると感じた。", 3),
        ("自動チェックイン機が故障しており、スタッフに案内されても対応が遅く、長時間並ぶ羽目になった。もっと機器のメンテナンスに気を使ってほしい。", 3)
    ],
    "positive": [
        ("初めての空港で少し不安だったが、スタッフが非常に親切で、スムーズに案内してくれた。わかりやすく、迅速に対応してくれたので、安心してフライトを楽しめた。", 4),
        ("旅行後、空港に到着した際、荷物受け取りのスタッフが素早く手続きをしてくれ、あっという間に荷物を受け取ることができた。非常にストレスの少ない体験だった。", 5),
        ("長時間の待機中、スタッフがラウンジのサービスをしっかりと説明してくれて、快適な時間を過ごせた。ドリンクも豊富で、リラックスできた。", 4),
        ("機内食のリクエストをしていたが、スタッフはしっかり覚えていて、私が希望した食事を提供してくれた。こうした配慮が本当にありがたかった。", 4),
        ("初めての空港で迷子になってしまったが、スタッフが親切に対応してくれて、無事に目的地に辿り着けた。すごく助かりました。", 4),
        ("フライトが遅れた際、スタッフが待機中の案内をしっかり行ってくれ、遅延の原因や最新情報を逐一提供してくれた。こうした細やかな配慮がありがたかった。", 5),
        ("空港内で小さな問題が発生したとき、スタッフがすぐに気づき、声をかけてくれた。こうした対応に感動し、安心感を得ることができた。", 4),
        ("手荷物預けの際、スタッフが笑顔で対応してくれて、そのまま気持ちよく飛行機に向かうことができた。こうした接客が本当に大切だと思った。", 4),
        ("空港内でちょっとした質問をした際、スタッフがとても親切に教えてくれた。わかりやすい案内と、優しい態度に安心できた。", 3),
        ("旅行中に予期しない問題が起こったが、スタッフが迅速かつ適切な解決策を提案してくれて、問題がスムーズに解決した。", 5),
        ("空港内での案内がとても分かりやすく、迷わずゲートに到着できた。スタッフがあらかじめ詳細な情報を提供してくれたので、安心してフライトを楽しめた。", 4),
        ("空港のラウンジで、スタッフが非常に快適に過ごせるように配慮してくれ、リラックスできる時間を提供してくれた。素晴らしいサービスだった。", 4),
        ("長時間の待機中、スタッフが気配りを見せ、飲み物のサービスをしてくれた。少しの配慮が大きな安心感に繋がった。", 3),
        ("フライト後、空港スタッフが私の荷物を早急に手配してくれて、他の乗客と比べてスムーズに荷物を受け取れた。スタッフの気配りが素晴らしかった。", 4),
        ("空港内の案内所での対応が非常に親切で、私の質問に丁寧に答えてくれた。安心して旅行を進められた。", 3),
        ("スタッフの対応が早く、乗り継ぎの案内も親切で、非常にスムーズに移動できた。余計なストレスを感じることなく、目的地に到着できた。", 4),
        ("タクシーサービスで、スタッフが迅速に手配してくれて、空港から目的地まで快適に移動できた。非常に便利だった。", 4),
        ("空港のスタッフが、初めて利用した空港で困っていたとき、すぐに声をかけて案内してくれたので、非常に助かった。", 4),
        ("空港内の清掃が行き届いており、快適に過ごせた。スタッフがすぐに気づき、テーブルを整理してくれるなど、細やかな気配りが感じられた。", 3)
    ]
}

# create dataframe
df = create_reviews(
    review_category = "地上サービス",               # テーマ
    review_items    = review_items,               # レビューテキスト + スコア
    rating_col      = "rating_ground_service",    # カラム名
    start_id        = current_cnt + 1             # idの開始番号
)

# DataFrame → Spark
s_ground_df = spark.createDataFrame(df)

# カウントアップ
# current_cnt = 0     # 初期化
current_cnt += s_ground_df.count()

# print
print(f"単体カウント: {s_ground_df.count()}")
print(f"ここまでの総カウント: {current_cnt}")
print(s_ground_df.columns)
if DISPLAY_ON:
    display(s_ground_df)

##### 2-8.最終レビュー生成に向けた前処理
ここでは次のセルの事前準備を行います。  
ここまで生成したカテゴリのレビューデータをまとめて、3つのカラムoriginal_1、original_2、original_3に展開します。  
次のセルでは、ai_queryで3つのカラムのレビューを組み合わせて1つの顧客レビューを生成します。  

注意：評価が存在しない（=null）カテゴリの評価は3~4の範囲で埋めます(レビューコメントの有無に関わらず評価はするため)

In [0]:
# ------------------------------------------------------------
# 前準備：共通列名の定義と「評価列」を NULL 埋めでそろえる関数
# ------------------------------------------------------------
from pyspark.sql import functions as F
from datetime import datetime, timedelta
import random

RATING_COLS = [
    "rating_ife",
    "rating_seat_comfort",
    "rating_food_beverages",
    "rating_value_for_money",
    "rating_wifi_connectivity",
    "rating_cabin_staff_service",
    "rating_ground_service"
]

def to_common(df, own_col):
    """
    各カテゴリ DataFrame を共通スキーマに変換する。
    """
    base_cols = ["id", "sentiment", "review_category", "review_text"]
    exprs = base_cols + [
        F.col(c).alias(c) if c == own_col else F.lit(None).cast("int").alias(c)
        for c in RATING_COLS
    ]
    return df.select(*exprs)

# ------------------------------------------------------------
# 7 つのカテゴリ DataFrame を共通化 → unionByName で結合
# ------------------------------------------------------------
ife_common    = to_common(s_ife_df,   "rating_ife")
seat_common   = to_common(s_seat_df,  "rating_seat_comfort")
foods_common  = to_common(s_foods_df, "rating_food_beverages")
value_common  = to_common(s_value_df, "rating_value_for_money")
wifi_common   = to_common(s_wifi_df,  "rating_wifi_connectivity")
staff_common  = to_common(s_staff_df, "rating_cabin_staff_service")
ground_common = to_common(s_ground_df,"rating_ground_service")

result_df = (
    ife_common
      .unionByName(seat_common,  allowMissingColumns=True)
      .unionByName(foods_common, allowMissingColumns=True)
      .unionByName(value_common, allowMissingColumns=True)
      .unionByName(wifi_common,  allowMissingColumns=True)
      .unionByName(staff_common, allowMissingColumns=True)
      .unionByName(ground_common,allowMissingColumns=True)
)

# ------------------------------------------------------------
# original_1〜3 ＋ 7 種評価列をまとめて出力行に組み立て
# ------------------------------------------------------------
used_ids = set()
output_rows = []

while result_df.count() > len(used_ids):
    remaining_df = result_df.filter(~result_df["id"].isin(used_ids))
    if remaining_df.count() == 0:
        break

    n = random.choice([2, 3])
    selected = remaining_df.orderBy(F.rand()).limit(n).collect()

    used_ids.update([r.id for r in selected])

    texts = [r.review_text for r in selected] + [None] * 3
    texts = texts[:3]

    # --- 評価列を取り出し、NULL の場合は 3〜4 で補完 ---
    row_scores = {}
    for col in RATING_COLS:
        # 評価が付いている行があればその値を使用
        val = next((getattr(r, col) for r in selected if getattr(r, col) is not None), None)
        # 無ければ 3〜4 のランダム値
        row_scores[col] = val if val is not None else random.randint(3, 4)

    # --- レビュー投稿日をランダム生成 ---
    rnd_day = datetime.now() - timedelta(days=random.randint(1, 365))　# 実行当日の「前日」から 365 日前まで

    # --- レコード生成 ---
    output_rows.append(
        (
            len(output_rows) + 1,
            rnd_day.strftime("%Y-%m-%d"),
            texts[0],
            texts[1],
            texts[2],
            row_scores["rating_ife"],
            row_scores["rating_seat_comfort"],
            row_scores["rating_food_beverages"],
            row_scores["rating_value_for_money"],
            row_scores["rating_wifi_connectivity"],
            row_scores["rating_cabin_staff_service"],
            row_scores["rating_ground_service"]
        )
    )

# ------------------------------------------------------------
# Spark DataFrame 化 & TempView作成
# ------------------------------------------------------------
schema_cols = [
    "id", "review_date", "original_1", "original_2", "original_3"
] + RATING_COLS

pre_reviews_df = spark.createDataFrame(output_rows, schema_cols)

# review_date を DATE 型に変換
pre_reviews_df = pre_reviews_df.withColumn(
    "review_date",
    F.to_date("review_date", "yyyy-MM-dd")
)

# TempViewテーブル作成
pre_reviews_df.createOrReplaceTempView("v_pre_reviews")

# print
print(pre_reviews_df.count())
print(pre_reviews_df.columns)
if DISPLAY_ON:
    display(pre_reviews_df)

##### 2-9.LLMでレビューデータ生成
ここではai_queryでLLMを呼び出し、本物に近い顧客レビューを生成します。  
3つのカラム（original_1, original_2, original_3）に設定したレビューを組み合わせて1つの顧客レビューを生成します。  
テーマや感情の異なるレビューを混ぜることで、実際に会員が自由に記述したレビュー文章に近いレビューデータを生成します。

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, expr
from pyspark.sql.types import StringType

# プロンプト生成関数
def create_prompt(original_1, original_2, original_3):
    """
    複数レビューをプロンプトに変換
    """
    prompt = f"""
    [指示]
    航空サービスの顧客レビューのVoC分析のデモデータが必要です。
    [内容]に顧客レビューを複数渡します。これら全てまとめて1人の会員が書いたレビューとして自然に聞こえるように繋げてください。
    顧客のリアルな航空サービス体験としてリアルな内容となるように、その顧客自身の背景についても想像を膨らませてください。
    なお[注意]は厳守です。

    [内容]
    {original_1}
    {original_2}
    {original_3}

    [注意]
    * 補足一切不要(そのまま分析に使うので)
    * 「」や""などで括らないこと（そのまま分析に使うので）
    * 300文字程度
    """
    return prompt

# UDFとして登録
create_prompt_udf = udf(create_prompt, StringType())

# ai_queryを使って、1人の顧客レビューを生成
# ai_queryをPythonから適用します。exprを使うことでPythonコード内でSQL関数を利用可能です。
reviews_df = spark.sql(f"""
WITH prompts AS (
  SELECT
    id AS review_id,
    original_1,
    original_2,
    original_3,
    review_date,
    -- プロンプトをSQLで生成
    concat_ws('\n',
      '[指示] 航空サービスの顧客レビューのVoC分析のデモデータが必要です。',
      '[内容]に顧客レビューを複数渡します。これら全てまとめて1人の会員が書いたレビューとして自然に聞こえるように繋げてください。',
      '顧客のリアルな航空サービス体験としてリアルな内容となるように、その顧客自身の背景についても想像を膨らませてください。なお[注意]は厳守です。',
      '[内容]',
      coalesce(original_1, ''),
      coalesce(original_2, ''),
      coalesce(original_3, ''),
      '[注意]'
      '\n * 補足一切不要(そのまま分析に使うので)'
      '\n * 「」や""などで括らないこと（そのまま分析に使うので）'
      '\n * 300文字程度'
    ) AS prompt,
    rating_ife,
    rating_seat_comfort,
    rating_food_beverages,
    rating_value_for_money,
    rating_wifi_connectivity,
    rating_cabin_staff_service,
    rating_ground_service
  FROM v_pre_reviews
  -- WHERE id IN (1,2,3,4,5)    -- テスト用に5件だけ
)
SELECT
  review_id,
  original_1,
  original_2,
  original_3,
  review_date,
  prompt,
  -- レビューを生成
  ai_query(
      -- 'databricks-claude-3-7-sonnet',
      'databricks-llama-4-maverick',
      prompt,
      failOnError => False
  ).result AS review,
  rating_ife,
  rating_seat_comfort,
  rating_food_beverages,
  rating_value_for_money,
  rating_wifi_connectivity,
  rating_cabin_staff_service,
  rating_ground_service
FROM prompts
""")

# 必要なカラムを取得します
reviews_with_rating_df = reviews_df.select(
  "review_id",
  "review_date",
  # "original_1",
  # "original_2",
  # "original_3",
  # "prompt",
  "review",
  "rating_ife",
  "rating_seat_comfort",
  "rating_food_beverages",
  "rating_value_for_money",
  "rating_wifi_connectivity",
  "rating_cabin_staff_service",
  "rating_ground_service"
)

# TempViewテーブル作成
reviews_with_rating_df.createOrReplaceTempView("v_reviews_with_rating")

# print
print(reviews_with_rating_df.count())
print(reviews_with_rating_df.columns)
if DISPLAY_ON:
  display(reviews_with_rating_df)

##### 2-10.総合評価を生成
ここではai_queryでLLMを呼び出し、前のセルで生成した顧客レビューに総合評価（5段階評価）をつけます。

In [0]:
# 直前セルで作った顧客レビューに、ai_queryを介してLLM で「総合評価」を付与します
reviews_with_overall_rating_df = spark.sql("""
WITH prompts AS (
    SELECT
        review_id,
        review_date,
        review,
        concat_ws('\n',
        '[指示] 次の顧客レビューを 1 から 5 の整数で総合評価してください。',
        '[内容]',
        coalesce(review, ''),
        '[注意]',
        '\n * 必ず数値のみを返してください。（そのまま数値型として扱うため）'
        ) AS prompt,
        rating_ife,
        rating_seat_comfort,
        rating_food_beverages,
        rating_value_for_money,
        rating_wifi_connectivity,
        rating_cabin_staff_service,
        rating_ground_service
    FROM {reviews_with_rating}
)
SELECT
    review_id,
    review_date,
    review,
    prompt,
    -- レビューを生成
    CAST(
        ai_query(
            -- 'databricks-claude-3-7-sonnet',
            'databricks-llama-4-maverick',
            prompt,
            failOnError => FALSE
        ).result AS INT
    ) AS rating_overall,
    rating_ife,
    rating_seat_comfort,
    rating_food_beverages,
    rating_value_for_money,
    rating_wifi_connectivity,
    rating_cabin_staff_service,
    rating_ground_service
FROM prompts
""", reviews_with_rating=reviews_with_rating_df)

# print
print(reviews_with_overall_rating_df.count())
print(reviews_with_overall_rating_df.columns)
if DISPLAY_ON:
    display(reviews_with_overall_rating_df)

##### 2-11. 推奨意向の追加
ここでは総合評価として「Bricks Airwaysを推奨するか？」に対する回答（True / FALSE）を生成します。

In [0]:
# 直前セルで作った顧客レビューに、ai_queryを介してLLM で「総合評価」を付与します
reviews_with_is_recommended_df = spark.sql("""
WITH prompts AS (
    SELECT
        review_id,
        review_date,
        review,
        concat_ws('\n',
        '[指示] 次の航空会社の顧客レビューを総合評価して、この航空会社を他者におすすめできるか否か？をTRUE or FALSE回答してください。',
        '[内容]',
        coalesce(review, ''),
        '[注意]',
        '\n * 必ずTRUE / FALSEのみ返してください。（そのままBOOLEAN型として扱うため）'
        ) AS prompt,
        rating_overall,
        rating_ife,
        rating_seat_comfort,
        rating_food_beverages,
        rating_value_for_money,
        rating_wifi_connectivity,
        rating_cabin_staff_service,
        rating_ground_service
    FROM {reviews_with_overall_rating}
)
SELECT
    review_id,
    review_date,
    review,
    prompt,
    -- レビューを生成
    CAST(
        ai_query(
            'databricks-llama-4-maverick',
            prompt,
            failOnError => FALSE
        ).result AS BOOLEAN
    ) AS is_recommended,
    rating_overall,
    rating_ife,
    rating_seat_comfort,
    rating_food_beverages,
    rating_value_for_money,
    rating_wifi_connectivity,
    rating_cabin_staff_service,
    rating_ground_service
FROM prompts
""", reviews_with_overall_rating=reviews_with_overall_rating_df)

# print
print(reviews_with_is_recommended_df.count())
print(reviews_with_is_recommended_df.columns)
if DISPLAY_ON:
    display(reviews_with_is_recommended_df)

##### 2-12. VoC分析用に不足カラムの追加
ここでは前のセルで生成した顧客レビューデータに、会員ID、航空機ID、シートタイプ、区間ID、航空機IDを追加します。  
これでVoC分析に用いる生データ by LLM の準備が完了です。

In [0]:
# ============================================================
# 追加カラムの仕様
# ============================================================
import random
from datetime import datetime, timedelta
from pyspark.sql import functions as F, Window
from pyspark.sql.types import IntegerType, StringType, DateType

# --------------------
# パラメータ
# --------------------
SEAT_DISTRIB  = [("First Class", 0.15),
                 ("Business Class", 0.20),
                 ("Premium Economy", 0.35),
                 ("Economy Class", 0.30)]

ROUTE_IDS = ["NRT-CDG","NRT-LHR","HND-LHR","HND-CTS","HND-ITM","HND-SFO","NRT-SIN"]
FLIGHT_IDS = ["A390-100","A425-900","A430-200","A392-300","A450-800","A470-900",
              "B830-10","B812-6","B845-9","B905-12",
              "E180-E2","E190-E2X","E260-E3",
              "S360-TP","S410-XLR","S385-TP","S450-XLR"]

# ============================================================
# user_id を作成 (1 回=85%, 2 回=10%, 3 回=5%)
# ============================================================
from math import ceil
total_reviews = reviews_with_is_recommended_df.count()

def gen_user_ids(n_reviews:int):
    user_ids = []
    uid = 1
    remain = n_reviews
    while remain > 0:
        r = random.random()
        if r < 0.85:
            k = 1
        elif r < 0.95:
            k = 2
        else:
            k = 3
        k = min(k, remain)        # 残数を超えない
        user_ids.extend([uid]*k)  # 同じ user_id を k 回
        uid += 1
        remain -= k
    random.shuffle(user_ids)      # 行との対応は後で window で付与
    return user_ids

user_id_list = gen_user_ids(total_reviews)

# DataFrame(row_num, user_id) を作成
user_id_df = spark.createDataFrame(
    list(enumerate(user_id_list, start=1)),
    ["row_num","user_id"]
)

# ============================================================
# reviews_with_overall_rating_df に追加列を付与
# ============================================================
w = Window.orderBy("review_id")
base_df = reviews_with_is_recommended_df.withColumn("row_num", F.row_number().over(w))

# --- user_id を JOIN ---
base_df = base_df.join(user_id_df, on="row_num").drop("row_num")

# --- seat_type ---
def choose_by_prob(prob_list):
    r = random.random()
    acc = 0.0
    for val, p in prob_list:
        acc += p
        if r < acc:
            return val
    return prob_list[-1][0]

choose_seat = F.udf(lambda: choose_by_prob(SEAT_DISTRIB), StringType())
base_df = base_df.withColumn("seat_type", choose_seat())

# --- route_id / aircraft_id ---
pick_route   = F.udf(lambda: random.choice(ROUTE_IDS), StringType())
pick_aircraft= F.udf(lambda: random.choice(FLIGHT_IDS), StringType())
base_df = (base_df
           .withColumn("route_id",   pick_route())
           .withColumn("flight_id",pick_aircraft()))

# --- flight_date (review_date から 1〜3 日前) ---
def flight_date_udf(review_date):
    bias_rand = random.random()
    if bias_rand < 0.95:
        offset = 1
    elif bias_rand < 0.99:
        offset = 2
    else:
        offset = 3
    return (review_date - timedelta(days=offset)).strftime("%Y-%m-%d")

flight_date = F.udf(lambda d: flight_date_udf(d), StringType())
base_df = base_df.withColumn("flight_date", flight_date(F.col("review_date").cast("date")))

# ============================================================
# 最終テーブル
# ============================================================
final_cols = [
  "review_id",
  "user_id",
  "review_date",
  "review",
  "flight_id",
  "flight_date",
  "route_id",
  "seat_type",
  "is_recommended",
  "rating_overall",
  "rating_ife",
  "rating_seat_comfort",
  "rating_food_beverages",
  "rating_value_for_money",
  "rating_wifi_connectivity",
  "rating_cabin_staff_service",
  "rating_ground_service"
]

voc_raw_df = base_df.select(*final_cols)

# DATE 型に変換
voc_raw_df = voc_raw_df.withColumn(
    "flight_date",
    F.to_date("flight_date", "yyyy-MM-dd")
)

# テーブルに保存 - ここではデモ用にコメントアウトします。直接テーブル作成せず、CSV出力することにします。
# voc_raw_df.write.format("delta").mode("overwrite").saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.bz_reviews")

# print
print(f"レコード件数: {voc_raw_df.count()}")
print(voc_raw_df.columns)
if DISPLAY_ON:
    display(voc_raw_df)

##### 2-13. VolumeにCSV出力

In [0]:
# ---------- CSV 出力 ----------
out_path = f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/reviews.csv"
voc_raw_df.coalesce(1).toPandas().to_csv(out_path, index=False)
print(out_path)

In [0]:
print("顧客レビューのCSVファイルを作成しました！")